In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import json
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

name="aak_maindb"
user="aak_maindbusr"
pwd="AakDBUsr#2021"
host="52.117.247.165"
client="mysql"
port="3306"
db_string=client+"://"+user+":"+pwd+"@"+host+":"+port+"/"+name


#create connection
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
connect_args={'ssl':{'fake_flag_to_enable_tls': True},
             'port': 3306}

db_connection_str = db_string
db_connection = create_engine(db_connection_str,connect_args= connect_args)

In [2]:
def get_total_rows(table_name):
    sql="SELECT COUNT(*) as count FROM "+table_name
    df = pd.read_sql(sql, db_connection)
    return df['count'].values[0]

get_total_rows("acp_companies")

1226070

In [18]:
def get_company_url():
    sql="SELECT name,homepage_url FROM acp_companies"
    df = pd.read_sql(sql, db_connection)
    return df

df=get_company_url()
df.head()

,name,homepage_url
0,Wetpaint,http://www.wetpaint.com/
1,Zoho,https://www.zoho.com/
2,Digg,http://www.digg.com
3,Facebook,http://www.facebook.com
4,Omnidrive,http://www.omnidrive.com


In [19]:
df.shape

(1226070, 2)

In [21]:
import requests
import shutil
import base64


data={}
data['name']=[]
data['homepage_url']=[]
data['base64_img']=[]


base_url="https://logo.clearbit.com/"
with tqdm(total=len(df)) as pbar:
    for i,row in df.iterrows():
        url=row['homepage_url'].replace('http://','')
        url=url.replace('https://','')
        new_url=base_url+url
        
        r = requests.get(url = new_url, stream=True)
        try:
            if r.status_code==200:
                with open('img.png', 'wb') as out_file:
                    shutil.copyfileobj(r.raw, out_file)
                data['name'].append(row['name'])
                data['homepage_url'].append(row['homepage_url'])

                with open('img.png', "rb") as image_file:
                    encoded_string = base64.b64encode(image_file.read())
                data['base64_img'].append(encoded_string)
        except Exception as e:
            print("EXC: ", e)
            pass
        pbar.update(1)

  0%|          | 228/1226070 [05:06<457:46:13,  1.34s/it]


KeyboardInterrupt: 

In [24]:
data

{'name': ['Wetpaint',
  'Zoho',
  'Digg',
  'Facebook',
  'Geni',
  'Fox Interactive Media',
  'Twitter',
  'StumbleUpon',
  'Scribd',
  'Slacker',
  'eBay',
  'MeetMoi',
  'Postini',
  'Li Ka Shing Foundation',
  'CBS',
  'Viacom',
  'Babelgum',
  'Plaxo',
  'Cisco',
  'Powerset',
  'Technorati',
  'AddThis',
  'OpenX',
  'Mahalo',
  'Kyte',
  'Time Warner',
  'Jingle Networks',
  'Hearst Communications',
  'STRANDS',
  'Ning Interactive',
  'LifeLock',
  'Wesabe',
  'Prosper Marketplace',
  'YouTube',
  'blogTV',
  'Justin.TV',
  'Ustream',
  'GrandCentral',
  'Tribune Media',
  'Gannett Co Inc',
  'Jobster',
  'Pownce',
  'Revision3',
  'Aggregate Knowledge',
  'CriticalMetrics',
  'ZenZui',
  'SodaHead',
  'HOT or NOT',
  'POPSUGAR',
  'NBCUniversal',
  'fring',
  'iSkoot',
  'EQO',
  'Sellaband',
  'Funny Or Die',
  'Steorn',
  'Surphace',
  'iContact',
  'blinkx',
  'Zlio',
  'Yelp',
  'Jaiku',
  'Tun3r',
  'Coghead',
  'Zooomr',
  'SideStep',
  'Yapta',
  'Dailymotion',
  'KickA

In [28]:
df1=pd.DataFrame(data)
df1.head()

,name,homepage_url,base64_img
0,Wetpaint,http://www.wetpaint.com/,b'iVBORw0KGgoAAAANSUhEUgAAAIAAAABgCAIAAABaGO0e...
1,Zoho,https://www.zoho.com/,b'iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAIAAABMXPac...
2,Digg,http://www.digg.com,b'iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAIAAABMXPac...
3,Facebook,http://www.facebook.com,b'iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHL...
4,Geni,http://www.geni.com,b'iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAIAAABMXPac...


In [29]:
df1.shape

(167, 3)